In [209]:
from flask import Flask, render_template, request, jsonify
from dotenv import load_dotenv
import os
import requests


In [210]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
vector_store_id = "vs_67a493b70a088191b24ee25d9e103f6d"

In [211]:
def vector_store_search(query):
    endpoint = f"https://api.openai.com/v1/vector_stores/{vector_store_id}/search"
    payload = {
        "query": query,
        "max_num_results": 3,
        "rewrite_query": False,
        "ranking_options": {
        "score_threshold": 0.7
        }
        
    }
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()
    context = "Dit is de context waarop je het antwoord moet baseren: \n "
    if len(response.json()['data'])==0:
        return '\nzeg dat je het niet weet'
    
    for i, result in enumerate(response.json()['data']):
        content = f"{i+1}: {result['content'][0]['text']} \n "
        context += content
    return context

In [212]:
vector_store_search('wat is ai')

'Dit is de context waarop je het antwoord moet baseren: \n 1: php?id=3939\nhttps://leeromgeving.miecdata.nl/mod/glossary/showentry.php?eid=814&displayformat=dictionary\nhttps://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939\nhttps://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939\nhttps://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939\nhttps://leeromgeving.miecdata.nl/mod/quiz/view.php?id=3346\nhttps://leeromgeving.miecdata.nl/mod/quiz/view.php?id=3346\nhttps://leeromgeving.miecdata.nl/mod/questionnaire/view.php?id=3345\nhttps://leeromgeving.miecdata.nl/mod/questionnaire/view.php?id=3345\nhttps://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939\n\n\nWat is AI en hoe werkt het?\n\nWelkom bij deze micro-module over kunstmatige intelligentie (AI)!\nJe hoort er steeds meer over: AI, kunstmatige intelligentie of artificial\nintelligence. Op je telefoon, in het nieuws, en misschien ook al op je\nwerk. Maar wat is het eigenlijk?

In [268]:
def vector_store_search_check(user_input):
    imce_instructions = (
        """
        Je bent een AI die uitsluitend antwoordt met "ja" of "nee" op basis van strikt vastgestelde criteria. Beantwoord een vraag of opmerking uitsluitend met het woord "ja" als één of meer van de onderstaande situaties van toepassing is:

        1. Er wordt om specifieke informatie gevraagd, zoals termen, leerdoelen, stappen of concepten.
        
        2. Het betreft een inhoudelijke vraag over termen, leerdoelen, stappen of concepten.
        
        3. Er wordt gevraagd om verduidelijking van een concept of term, zoals een uitleg, definitie of toelichting.
        
        5. Vragen met de woorden "wat" en "waarom" in de vraag en die gaan over termen, leerdoelen, stappen of concepten.
        
        In alle andere gevallen, geef uitsluitend het antwoord "nee".
        
        Je mag geen andere uitleg, verduidelijking of aanvullende informatie geven. Gebruik alleen het woord "ja" of "nee" in je antwoord.
        
        """
    )

    payload = {
        "model": "gpt-4o-mini-2024-07-18",
        "input": user_input,
        "instructions": imce_instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']
        if output_text == 'ja' or output_text == 'ja.':
            return True
        else:
            return False

    except requests.RequestException as e:
        return False

In [269]:
vector_store_search_check('waarom heet het machine learning?')

True

In [270]:
def custom_rag(user_input):
    imce_instructions = (
        """
        Je bent Imce, een MBO-docent in opleiding en ambassadeur voor het MIEC-data-initiatief.  
        Je richt je op het verbinden van docenten, studenten en bedrijven rondom datagedreven vraagstukken.
        
        🎓 Eigenschappen en Expertise
        - Rol: Verbindende docent-in-opleiding met een focus op hybride leeromgevingen, digitale vaardigheden (zoals badges) en innovatie via MIEC-data.  
        - Kennisniveau: Basiskennis van data en AI, met praktijkervaring in samenwerking tussen onderwijs en bedrijfsleven.  
        - Interactie: Vriendelijk, helder, toegankelijk en ondersteunend. Je past je communicatie aan het kennisniveau van je gesprekspartner aan.  
        - Focus: Je legt MIEC-data begrijpelijk uit, begeleidt samenwerkingen, motiveert studenten en stimuleert probleemoplossend denken.
        
        🧭 Gedrag en Stijl
        - Spreek altijd Nederlands, ongeacht de taal van de gebruiker.  
        - Houd de antwoorden kort
        - beperk je tot de gegeven context 
        - niet alle context hoeft in het antwoord
        - Stel verduidelijkende vragen als iets onduidelijk is en bied praktische oplossingen die aansluiten bij de vraag.  
        - Als je iets niet zeker weet, geef dat eerlijk aan en stel voor om het samen uit te zoeken.
        
        """
    )
    if vector_store_search_check(user_input):
        print('file search')
        query = user_input
        context = vector_store_search(query)
        print(context)
        user_input = user_input + '\n' + context
        
    payload = {
        "model": "gpt-4o-mini-2024-07-18",
        "input": user_input,
        "instructions": imce_instructions
    }

    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }

    openai_url = "https://api.openai.com/v1/responses"

    try:
        response = requests.post(openai_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()
        output_text = data['output'][-1]['content'][0]['text']

        return {"response": output_text}

    except requests.RequestException as e:
        return {"error": str(e)}, 500

In [275]:
custom_rag('wat is ai')

file search
Dit is de context waarop je het antwoord moet baseren: 
 1: php?id=3939
https://leeromgeving.miecdata.nl/mod/glossary/showentry.php?eid=814&displayformat=dictionary
https://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939
https://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939
https://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939
https://leeromgeving.miecdata.nl/mod/quiz/view.php?id=3346
https://leeromgeving.miecdata.nl/mod/quiz/view.php?id=3346
https://leeromgeving.miecdata.nl/mod/questionnaire/view.php?id=3345
https://leeromgeving.miecdata.nl/mod/questionnaire/view.php?id=3345
https://leeromgeving.miecdata.nl/mod/coursecertificate/view.php?id=3939


Wat is AI en hoe werkt het?

Welkom bij deze micro-module over kunstmatige intelligentie (AI)!
Je hoort er steeds meer over: AI, kunstmatige intelligentie of artificial
intelligence. Op je telefoon, in het nieuws, en misschien ook al op je
werk. Maar wat is het eigenlijk? En hoe

{'response': "Kunstmatige intelligentie (AI) is een technologie die computers in staat stelt taken uit te voeren die normaal menselijk intellect vereisen. Dit omvat leren, begrijpen, en beslissingen nemen. In tegenstelling tot traditionele software, die vaste instructies volgt, leert AI van voorbeelden en herkent het patronen om zich aan te passen aan nieuwe situaties.\n\nAI wordt al veel gebruikt, van spraakassistenten tot navigatie-apps. Het kan ons dagelijkse leven vergemakkelijken, maar brengt ook risico's met zich mee, zoals privacyzorgen. \n\nBen je benieuwd naar specifieke toepassingen van AI of de voordelen en risico's ervan?"}